In [1]:
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from pyspark.sql import SparkSession
from pysparkling import H2OContext
import h2o

In [2]:
# input_1 : Window_size = 5
train_folder = 'input_1/train'
valid_folder = 'input_1/valid'
n_fold = 5
n_digits = 4
seed = 1
max_runtime_secs = 1000

# input_2 : Window_size - 10
# train_folder = 'input_2/train'
# valid_folder = 'input_2/valid'
# n_fold = 5
# n_digits = 3
# seed = 2
# max_runtime_secs = 1500

In [3]:
ss = SparkSession.builder.config("spark.executor.memory", "12g")\
    .config("spark.driver.memory", "12g")\
    .config("spark.executor.cores", 6)\
    .config('spark.executor.instances', 5).getOrCreate()

ss.sparkContext.setLogLevel('OFF')

In [4]:
hc = H2OContext.getOrCreate()

Connecting to H2O server at http://192.168.1.7:54323 ... successful.


H2O_cluster_uptime:,09 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,26 days
H2O_cluster_name:,sparkling-water-yuefeiwang_local-1614535132510
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,10.54 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,6
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.32.0.4-1-3.0
 * H2O name: sparkling-water-yuefeiwang_local-1614535132510
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,192.168.1.7,54321)
  ------------------------

  Open H2O Flow in browser: http://192.168.1.7:54323 (CMD + click in Mac OSX)

    


# Question 1

In [5]:
# create spark dataframe
train_df = ss.read.parquet(train_folder).repartition(8).cache()
valid_df = ss.read.parquet(valid_folder).repartition(8).cache()

In [6]:
# convert spark dataframe to h2oFrame
train_h2o = hc.asH2OFrame(train_df, "train")
valid_h2o = hc.asH2OFrame(valid_df, "valid")

In [7]:
train_h2o['label'] = train_h2o['label'].asfactor()
valid_h2o['label'] = valid_h2o['label'].asfactor()

In [8]:
print(train_h2o.types)

{'features.0': 'real', 'features.1': 'real', 'features.2': 'real', 'features.3': 'real', 'features.4': 'real', 'features.5': 'real', 'features.6': 'real', 'features.7': 'real', 'features.8': 'real', 'features.9': 'real', 'features.10': 'real', 'features.11': 'real', 'features.12': 'real', 'features.13': 'real', 'features.14': 'real', 'features.15': 'real', 'features.16': 'real', 'features.17': 'real', 'features.18': 'real', 'features.19': 'real', 'features.20': 'real', 'features.21': 'real', 'features.22': 'real', 'features.23': 'real', 'features.24': 'real', 'features.25': 'real', 'features.26': 'real', 'features.27': 'real', 'features.28': 'real', 'features.29': 'real', 'features.30': 'real', 'features.31': 'real', 'features.32': 'real', 'features.33': 'real', 'features.34': 'real', 'features.35': 'real', 'features.36': 'int', 'features.37': 'int', 'label': 'enum'}


In [9]:
print(valid_h2o.types)

{'features.0': 'real', 'features.1': 'real', 'features.2': 'real', 'features.3': 'real', 'features.4': 'real', 'features.5': 'real', 'features.6': 'real', 'features.7': 'real', 'features.8': 'real', 'features.9': 'real', 'features.10': 'real', 'features.11': 'real', 'features.12': 'real', 'features.13': 'real', 'features.14': 'real', 'features.15': 'real', 'features.16': 'real', 'features.17': 'real', 'features.18': 'real', 'features.19': 'real', 'features.20': 'real', 'features.21': 'real', 'features.22': 'real', 'features.23': 'real', 'features.24': 'real', 'features.25': 'real', 'features.26': 'real', 'features.27': 'real', 'features.28': 'real', 'features.29': 'real', 'features.30': 'real', 'features.31': 'real', 'features.32': 'real', 'features.33': 'real', 'features.34': 'real', 'features.35': 'real', 'features.36': 'int', 'features.37': 'int', 'label': 'enum'}


# Question 2

In [10]:
predictors = train_h2o.names[:]
response = 'label'
predictors.remove(response)

In [11]:
print(predictors)

['features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'features.13', 'features.14', 'features.15', 'features.16', 'features.17', 'features.18', 'features.19', 'features.20', 'features.21', 'features.22', 'features.23', 'features.24', 'features.25', 'features.26', 'features.27', 'features.28', 'features.29', 'features.30', 'features.31', 'features.32', 'features.33', 'features.34', 'features.35', 'features.36', 'features.37']


# Question 3

In [13]:
model_xg = H2OXGBoostEstimator(max_runtime_secs=max_runtime_secs,
                               nfolds=n_fold,
                               seed=seed)
model_xg.train(x=predictors,
               y=response,
               training_frame=train_h2o,
               validation_frame=valid_h2o
            )

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [14]:
print(round(model_xg.auc(valid=True),n_digits))  # print ROC of the validation set

0.8658


In [15]:
print(model_xg.confusion_matrix(valid=True))  # print confusion matrix for the validation set


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.37029208699291444: 


,,0,1,Error,Rate
0,0,678728.0,170815.0,0.2011,(170815.0/849543.0)
1,1,80056.0,249863.0,0.2427,(80056.0/329919.0)
2,Total,758784.0,420678.0,0.2127,(250871.0/1179462.0)


# Question 4

In [24]:
model_dl =H2ODeepLearningEstimator(nfolds=n_fold, 
                                   variable_importances=True,
                                   loss='Automatic',
                                   max_runtime_secs=max_runtime_secs,
                                   seed=seed)
model_dl.train(x=predictors,
               y=response,
               training_frame=train_h2o,
               validation_frame=valid_h2o)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [25]:
print(round(model_dl.auc(valid=True),n_digits)) 

0.9448


In [26]:
print(model_dl.confusion_matrix(valid=True)) 


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.36022527328601067: 


,,0,1,Error,Rate
0,0,739742.0,109801.0,0.1292,(109801.0/849543.0)
1,1,43432.0,286487.0,0.1316,(43432.0/329919.0)
2,Total,783174.0,396288.0,0.1299,(153233.0/1179462.0)


In [27]:
ss.stop()